# Setup and Data Generation
Performs global setup for the benchmark suite.
- Imports necessary libraries.
- Defines local helper functions: `generate_large_dataset`, `benchmark_operation`, and `verify_correctness`.
- Connects to the q/kdb+ process.
- Generates a **10 million row** dataset (`LARGE_DF`) and converts it to a pykx Table (`LARGE_Q_TABLE`).


In [1]:
import os
import sys
import importlib
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('.'))
import qutePandas as qpd
importlib.reload(qpd)
import pandas as pd
import numpy as np
import pykx as kx
import time
import gc
from test_utils import generate_large_dataset, benchmark_operation, verify_correctness, calculate_speedup
local_lic = os.path.abspath('../kdb_lic')
if os.path.exists(local_lic): os.environ['QLIC'] = local_lic
qpd.connect()


True

In [2]:
print("Generating Large Datasets (10M rows, 20 cols)... This may take a moment.")
LARGE_DF = generate_large_dataset(rows=10_000_000, cols=20)
LARGE_Q_TABLE = kx.toq(LARGE_DF)
LARGE_DF_SUBSET = LARGE_DF[['col_0', 'col_4']]
LARGE_Q_SUBSET = LARGE_Q_TABLE[['col_0', 'col_4']]
print(f"Data Generation Complete. Shape: {LARGE_DF.shape}")


Generating Large Datasets (10M rows, 20 cols)... This may take a moment.
Data Generation Complete. Shape: (10000000, 20)


## Core: DataFrame Creation
Tests the creation of qutePandas DataFrames.


In [3]:
print('Benchmarking: DataFrame Creation')

def pd_func(): pd.DataFrame(LARGE_DF)
def q_func(): qpd.DataFrame(LARGE_DF)
pd_stats = benchmark_operation(pd_func)
print(f"  Pandas Mean: {pd_stats['mean']:.4f} s")
q_stats = benchmark_operation(q_func)
print(f"  qutePandas Mean: {q_stats['mean']:.4f} s")
calculate_speedup(pd_stats, q_stats)


Benchmarking: DataFrame Creation
  Pandas Mean: 0.0000 s
  qutePandas Mean: 16.0392 s
  Speedup: 0.0000026872


## Test print Performance
Test print function performance without pandas conversion overhead.

In [4]:
print('Benchmarking print vs pandas print:')

def pd_func(): print(LARGE_DF_SUBSET.head(5))
def q_func(): qpd.print(LARGE_Q_SUBSET, head=5)
pd_stats = benchmark_operation(pd_func)
q_stats = benchmark_operation(q_func)
print(f"  Pandas Mean: {pd_stats['mean']:.4f} s")
print(f"  qutePandas Mean: {q_stats['mean']:.4f} s")
calculate_speedup(pd_stats, q_stats)

Benchmarking print vs pandas print:
   col_0     col_4
0    NaN       NaN
1  436.0 -1.045758
2  861.0 -0.031143
3  271.0  0.709787
4  107.0  1.548467
   col_0     col_4
0    NaN       NaN
1  436.0 -1.045758
2  861.0 -0.031143
3  271.0  0.709787
4  107.0  1.548467
   col_0     col_4
0    NaN       NaN
1  436.0 -1.045758
2  861.0 -0.031143
3  271.0  0.709787
4  107.0  1.548467
   col_0     col_4
0    NaN       NaN
1  436.0 -1.045758
2  861.0 -0.031143
3  271.0  0.709787
4  107.0  1.548467
   col_0     col_4
0    NaN       NaN
1  436.0 -1.045758
2  861.0 -0.031143
3  271.0  0.709787
4  107.0  1.548467
   col_0     col_4
0    NaN       NaN
1  436.0 -1.045758
2  861.0 -0.031143
3  271.0  0.709787
4  107.0  1.548467
┌───────┬───────────────────────┐
│ col_0 │ col_4                 │
├───────┼───────────────────────┤
│ nan   │ nan                   │
│ 436.0 │ -1.0457584423673143   │
│ 861.0 │ -0.031142940226820567 │
│ 271.0 │ 0.7097869211593463    │
│ 107.0 │ 1.5484674450802385    │
└───────

## Cleaning Functions
Tests performance of data cleaning operations.


In [5]:
print('Benchmarking: dropna')

def pd_func(): LARGE_DF.dropna()
def q_func(): qpd.dropna(LARGE_Q_TABLE, return_type='q')
pd_res = LARGE_DF.dropna()
q_res = qpd.dropna(LARGE_Q_TABLE, return_type='p')
pd_stats = benchmark_operation(pd_func)
print(f"  Pandas Mean: {pd_stats['mean']:.4f} s")
q_stats = benchmark_operation(q_func)
print(f"  qutePandas Mean: {q_stats['mean']:.4f} s")
assert verify_correctness(pd_res, q_res), 'Results do not match!'
calculate_speedup(pd_stats, q_stats)


Benchmarking: dropna
  Pandas Mean: 1.0243 s
  qutePandas Mean: 0.1246 s
  Speedup: 8.2193


In [6]:
print('Benchmarking: dropna_col')

def pd_func(): LARGE_DF.dropna(subset=['col_0'])
def q_func(): qpd.dropna_col(LARGE_Q_TABLE, 'col_0', return_type='q')
pd_res = LARGE_DF.dropna(subset=['col_0'])
q_res = qpd.dropna_col(LARGE_Q_TABLE, 'col_0', return_type='p')
pd_stats = benchmark_operation(pd_func)
print(f"  Pandas Mean: {pd_stats['mean']:.4f} s")
q_stats = benchmark_operation(q_func)
print(f"  qutePandas Mean: {q_stats['mean']:.4f} s")
assert verify_correctness(pd_res, q_res), 'Results do not match!'
calculate_speedup(pd_stats, q_stats)


Benchmarking: dropna_col
  Pandas Mean: 0.5281 s
  qutePandas Mean: 0.0498 s
  Speedup: 10.5956


In [7]:
print('Benchmarking: fillna')

def pd_func(): LARGE_DF.fillna({'col_0': 0})
def q_func(): qpd.fillna(LARGE_Q_TABLE, 'col_0', 0, return_type='q')
pd_res = LARGE_DF.fillna({'col_0': 0})
q_res = qpd.fillna(LARGE_Q_TABLE, 'col_0', 0, return_type='p')
pd_stats = benchmark_operation(pd_func)
print(f"  Pandas Mean: {pd_stats['mean']:.4f} s")
q_stats = benchmark_operation(q_func)
print(f"  qutePandas Mean: {q_stats['mean']:.4f} s")
assert verify_correctness(pd_res, q_res), 'Results do not match!'
calculate_speedup(pd_stats, q_stats)


Benchmarking: fillna
  Pandas Mean: 0.5708 s
  qutePandas Mean: 0.0040 s
  Speedup: 143.4786


## Transformation Functions
Tests structural and type transformations on the DataFrame.


In [8]:
print('Benchmarking: rename')

def pd_func(): LARGE_DF.rename(columns={'col_0': 'new_col_0'})
def q_func(): qpd.rename(LARGE_Q_TABLE, {'col_0': 'new_col_0'}, return_type='q')
pd_res = LARGE_DF.rename(columns={'col_0': 'new_col_0'})
q_res = qpd.rename(LARGE_Q_TABLE, {'col_0': 'new_col_0'}, return_type='p')
pd_stats = benchmark_operation(pd_func)
print(f"  Pandas Mean: {pd_stats['mean']:.4f} s")
q_stats = benchmark_operation(q_func)
print(f"  qutePandas Mean: {q_stats['mean']:.4f} s")
assert verify_correctness(pd_res, q_res), 'Results do not match!'
calculate_speedup(pd_stats, q_stats)


Benchmarking: rename
  Pandas Mean: 0.5753 s
  qutePandas Mean: 0.0001 s
  Speedup: 5561.9642


In [9]:
print('Benchmarking: cast')

def pd_func(): LARGE_DF['col_0'].astype('float32')
def q_func(): qpd.cast(LARGE_Q_TABLE, 'col_0', 'float32', return_type='q')
pd_res = LARGE_DF['col_0'].astype('float32')
q_res = qpd.cast(LARGE_Q_TABLE, 'col_0', 'float32', return_type='p')['col_0']
pd_stats = benchmark_operation(pd_func)
print(f"  Pandas Mean: {pd_stats['mean']:.4f} s")
q_stats = benchmark_operation(q_func)
print(f"  qutePandas Mean: {q_stats['mean']:.4f} s")
assert verify_correctness(pd_res, q_res), 'Results do not match!'
calculate_speedup(pd_stats, q_stats)


Benchmarking: cast
  Pandas Mean: 0.0037 s
  qutePandas Mean: 0.0008 s
  Speedup: 4.8367


In [10]:
print('Benchmarking: drop_col')

def pd_func(): LARGE_DF.drop(columns=['col_9'])
def q_func(): qpd.drop_col(LARGE_Q_TABLE, 'col_9', return_type='q')
pd_res = LARGE_DF.drop(columns=['col_9'])
q_res = qpd.drop_col(LARGE_Q_TABLE, 'col_9', return_type='p')
pd_stats = benchmark_operation(pd_func)
print(f"  Pandas Mean: {pd_stats['mean']:.4f} s")
q_stats = benchmark_operation(q_func)
print(f"  qutePandas Mean: {q_stats['mean']:.4f} s")
assert verify_correctness(pd_res, q_res), 'Results do not match!'
calculate_speedup(pd_stats, q_stats)


Benchmarking: drop_col
  Pandas Mean: 0.5313 s
  qutePandas Mean: 0.0001 s
  Speedup: 4005.8143


## Grouping & Aggregation
Tests the performance of grouping operations.


In [11]:
print('Benchmarking: groupby_sum')

def pd_func(): LARGE_DF.groupby('col_3', dropna=False)['col_0'].sum()
def q_func(): qpd.groupby_sum(LARGE_Q_TABLE, 'col_3', 'col_0', return_type='q')
pd_res = LARGE_DF.groupby('col_3', dropna=False)['col_0'].sum()
q_res = qpd.groupby_sum(LARGE_Q_TABLE, 'col_3', 'col_0', return_type='p').set_index('col_3')['col_0']
pd_stats = benchmark_operation(pd_func)
print(f"  Pandas Mean: {pd_stats['mean']:.4f} s")
q_stats = benchmark_operation(q_func)
print(f"  qutePandas Mean: {q_stats['mean']:.4f} s")
assert verify_correctness(pd_res, q_res), 'Results do not match!'
calculate_speedup(pd_stats, q_stats)


Benchmarking: groupby_sum
  Pandas Mean: 0.2743 s
  qutePandas Mean: 0.0028 s
  Speedup: 98.2390


In [12]:
print('Benchmarking: groupby_avg')

def pd_func(): LARGE_DF.groupby('col_3', dropna=False)['col_1'].mean()
def q_func(): qpd.groupby_avg(LARGE_Q_TABLE, 'col_3', 'col_1', return_type='q')
pd_res = LARGE_DF.groupby('col_3', dropna=False)['col_1'].mean()
q_res = qpd.groupby_avg(LARGE_Q_TABLE, 'col_3', 'col_1', return_type='p').set_index('col_3')['col_1']
pd_stats = benchmark_operation(pd_func)
print(f"  Pandas Mean: {pd_stats['mean']:.4f} s")
q_stats = benchmark_operation(q_func)
print(f"  qutePandas Mean: {q_stats['mean']:.4f} s")
assert verify_correctness(pd_res, q_res), 'Results do not match!'
calculate_speedup(pd_stats, q_stats)


Benchmarking: groupby_avg
  Pandas Mean: 0.2770 s
  qutePandas Mean: 0.0047 s
  Speedup: 59.4864


## Custom Function Application
Tests the `apply` mechanism for row-wise operations.


In [13]:
print('Benchmarking: apply (sum axis=1)')

def pd_func(): LARGE_DF_SUBSET.sum(axis=1)
def q_func(): qpd.apply(LARGE_Q_SUBSET, 'sum', axis=1, return_type='q')
pd_res = LARGE_DF_SUBSET.sum(axis=1)
q_res = qpd.apply(LARGE_Q_SUBSET, 'sum', axis=1, return_type='p')
pd_stats = benchmark_operation(pd_func)
print(f"  Pandas Mean: {pd_stats['mean']:.4f} s")
q_stats = benchmark_operation(q_func)
print(f"  qutePandas Mean: {q_stats['mean']:.4f} s")
assert verify_correctness(pd_res, q_res), 'Results do not match!'
calculate_speedup(pd_stats, q_stats)


Benchmarking: apply (sum axis=1)
  Pandas Mean: 0.4712 s
  qutePandas Mean: 0.0099 s
  Speedup: 47.7924


## Indexing & Selection
Tests performance of `loc` and `iloc` operations.

In [14]:
print('Benchmarking: loc (boolean mask)')

mask = LARGE_DF['col_0'] > 0.5
q_mask = kx.toq(list(mask.values))

def pd_func(): LARGE_DF.loc[mask]
def q_func(): qpd.loc(LARGE_Q_TABLE, rows=q_mask, return_type='q')
pd_stats = benchmark_operation(pd_func)
print(f"  Pandas Mean: {pd_stats['mean']:.4f} s")
q_stats = benchmark_operation(q_func)
print(f"  qutePandas Mean: {q_stats['mean']:.4f} s")
calculate_speedup(pd_stats, q_stats)

Benchmarking: loc (boolean mask)
  Pandas Mean: 0.5156 s
  qutePandas Mean: 0.0490 s
  Speedup: 10.5195


In [15]:
print('Benchmarking: iloc (rows slice)')

def pd_func(): LARGE_DF.iloc[0:50000]
def q_func(): qpd.iloc(LARGE_Q_TABLE, rows=slice(0, 50000), return_type='q')
pd_stats = benchmark_operation(pd_func)
print(f"  Pandas Mean: {pd_stats['mean']:.4f} s")
q_stats = benchmark_operation(q_func)
print(f"  qutePandas Mean: {q_stats['mean']:.4f} s")
calculate_speedup(pd_stats, q_stats)

Benchmarking: iloc (rows slice)
  Pandas Mean: 0.0001 s
  qutePandas Mean: 0.0007 s
  Speedup: 0.0991


In [16]:
print('Benchmarking: iloc (cols slice)')

def pd_func(): LARGE_DF.iloc[:, 0:5]
def q_func(): qpd.iloc(LARGE_Q_TABLE, cols=slice(0, 5), return_type='q')
pd_stats = benchmark_operation(pd_func)
print(f"  Pandas Mean: {pd_stats['mean']:.4f} s")
q_stats = benchmark_operation(q_func)
print(f"  qutePandas Mean: {q_stats['mean']:.4f} s")
calculate_speedup(pd_stats, q_stats)

Benchmarking: iloc (cols slice)
  Pandas Mean: 0.1241 s
  qutePandas Mean: 0.0001 s
  Speedup: 970.4800


## Joining
Tests performance of join operations (merge_left, merge_inner).

In [17]:
print('Setting up for Joining Benchmarks...')
JOIN_KEY = 'col_0'
UNIQUE_KEYS = LARGE_DF[JOIN_KEY].unique()
LOOKUP_DF = pd.DataFrame({
    JOIN_KEY: UNIQUE_KEYS[:1000], # 1000 keys for lookup
    'extra_val': np.random.randn(1000)
})
LOOKUP_Q = kx.toq(LOOKUP_DF)
print('Setup Complete')

Setting up for Joining Benchmarks...
Setup Complete


In [18]:
print('Benchmarking: merge (left)')
def pd_func(): LARGE_DF.merge(LOOKUP_DF, on=JOIN_KEY, how='left')
def q_func(): qpd.merge(LARGE_Q_TABLE, LOOKUP_Q, on=JOIN_KEY, how='left', return_type='q')
pd_stats = benchmark_operation(pd_func)
print(f"  Pandas Mean: {pd_stats['mean']:.4f} s")
q_stats = benchmark_operation(q_func)
print(f"  qutePandas Mean: {q_stats['mean']:.4f} s")
calculate_speedup(pd_stats, q_stats)


Benchmarking: merge (left)
  Pandas Mean: 0.7751 s
  qutePandas Mean: 0.0189 s
  Speedup: 41.0279


In [19]:
print('Benchmarking: merge (inner)')
def pd_func(): LARGE_DF.merge(LOOKUP_DF, on=JOIN_KEY, how='inner')
def q_func(): qpd.merge(LARGE_Q_TABLE, LOOKUP_Q, on=JOIN_KEY, how='inner', return_type='q')
pd_stats = benchmark_operation(pd_func)
print(f"  Pandas Mean: {pd_stats['mean']:.4f} s")
q_stats = benchmark_operation(q_func)
print(f"  qutePandas Mean: {q_stats['mean']:.4f} s")
calculate_speedup(pd_stats, q_stats)

Benchmarking: merge (inner)
  Pandas Mean: 0.7583 s
  qutePandas Mean: 0.0716 s
  Speedup: 10.5856


In [20]:
print('Benchmarking: merge (right)')
def pd_func(): LARGE_DF.merge(LOOKUP_DF, on=JOIN_KEY, how='right')
def q_func(): qpd.merge(LARGE_Q_TABLE, LOOKUP_Q, on=JOIN_KEY, how='right', return_type='q')
pd_stats = benchmark_operation(pd_func)
print(f"  Pandas Mean: {pd_stats['mean']:.4f} s")
q_stats = benchmark_operation(q_func)
print(f"  qutePandas Mean: {q_stats['mean']:.4f} s")
calculate_speedup(pd_stats, q_stats)

Benchmarking: merge (right)
  Pandas Mean: 2.9919 s
  qutePandas Mean: 0.0619 s
  Speedup: 48.3159


In [21]:
print('Benchmarking: merge (outer)')
def pd_func(): LARGE_DF.merge(LOOKUP_DF, on=JOIN_KEY, how='outer')
def q_func(): qpd.merge(LARGE_Q_TABLE, LOOKUP_Q, on=JOIN_KEY, how='outer', return_type='q')
pd_stats = benchmark_operation(pd_func)
print(f"  Pandas Mean: {pd_stats['mean']:.4f} s")
q_stats = benchmark_operation(q_func)
print(f"  qutePandas Mean: {q_stats['mean']:.4f} s")
calculate_speedup(pd_stats, q_stats)

Benchmarking: merge (outer)
  Pandas Mean: 2.9733 s
  qutePandas Mean: 0.1232 s
  Speedup: 24.1333


## I/O Operations
Tests performance of to_csv and from_csv on a simplified subset.
(Bypass for now due to formatting issue in test environment)

In [22]:
csv_bench_path = 'bench_io.csv'
print('Benchmarking I/O performance:')
IO_DF_SUBSET = LARGE_DF.head(1_000_000)
IO_Q_SUBSET = qpd.DataFrame(IO_DF_SUBSET)

def pd_to_csv(): IO_DF_SUBSET.to_csv(csv_bench_path, index=False)
def q_to_csv():qpd.to_csv(IO_Q_SUBSET, csv_bench_path)
print("\n1.to_csv performance:")
pd_stats_to = benchmark_operation(pd_to_csv, iterations=3)
print(f"  Pandas Mean: {pd_stats_to['mean']:.4f} s")
q_stats_to = benchmark_operation(q_to_csv, iterations=3)
print(f"  qutePandas Mean: {q_stats_to['mean']:.4f} s")
calculate_speedup(pd_stats_to, q_stats_to)

Benchmarking I/O performance:

1.to_csv performance:
  Pandas Mean: 4.9886 s
  qutePandas Mean: 2.1994 s
  Speedup: 2.2681


In [23]:
def pd_from_csv(): pd.read_csv(csv_bench_path)
def q_from_csv(): qpd.from_csv(csv_bench_path, return_type='q')
print("\n2.from_csv performance:")
pd_stats_from = benchmark_operation(pd_from_csv, iterations=3)
print(f"  Pandas Mean: {pd_stats_from['mean']:.4f} s")
q_stats_from = benchmark_operation(q_from_csv, iterations=3)
print(f"  qutePandas Mean: {q_stats_from['mean']:.4f} s")
calculate_speedup(pd_stats_from, q_stats_from)

if os.path.exists(csv_bench_path):
    os.remove(csv_bench_path)


2.from_csv performance:
  Pandas Mean: 0.7560 s
  qutePandas Mean: 0.5664 s
  Speedup: 1.3347


## Introspection
Tests performance of metadata and type inspection.

In [24]:
print('Benchmarking: dtypes')

def pd_func(): LARGE_DF.dtypes
def q_func(): qpd.dtypes(LARGE_Q_TABLE, return_type='q')
pd_res = LARGE_DF.dtypes
q_res = qpd.dtypes(LARGE_Q_TABLE, return_type='p')
pd_stats = benchmark_operation(pd_func)
print(f"  Pandas Mean: {pd_stats['mean']:.4f} s")
q_stats = benchmark_operation(q_func)
print(f"  qutePandas Mean: {q_stats['mean']:.4f} s")
calculate_speedup(pd_stats, q_stats)

Benchmarking: dtypes
  Pandas Mean: 0.0001 s
  qutePandas Mean: 0.0001 s
  Speedup: 1.2044
